# XLSR- 53 model finetuned on Common voice german dataset

### XLSR-Wav2Vec2 is a speech model
1. It accepts a float array (raw waveform) of the speech signal.
2. The model was trained using connectionist temporal classification (CTC)
3. The model output has to be decoded using Wav2Vec2CTCTokenizer.

- In this notebook we have compared the Predicted and Reference Transcriptions

In [1]:
# importing Libraries
import torch
import torchaudio
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

In [2]:
#Loading the data, processor and model
test_dataset = load_dataset("common_voice", "de", split="test[:4%]")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-xlsr-53-german")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-xlsr-53-german")

Reusing dataset common_voice (/home/iqbalc/.cache/huggingface/datasets/common_voice/de/6.1.0/a1dc74461f6c839bfe1e8cf1262fd4cf24297e3fbd4087a711bd090779023a5e)


In [3]:
#sampling the audio to 16000Hz
resampler = torchaudio.transforms.Resample(48_000, 16_000)

In [4]:
# Preprocessing the datasets.
# We need to read the aduio files as arrays

def speech_file_to_array_fn(batch):
        speech_array, sampling_rate = torchaudio.load(batch["path"])
        batch["speech"] = resampler(speech_array).squeeze().numpy()
        return batch


In [13]:
test_dataset = test_dataset.map(speech_file_to_array_fn)
inputs = processor(test_dataset[:10]["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

  0%|          | 0/624 [00:00<?, ?ex/s]

In [14]:
# Store logits (non-normalized predictions)
with torch.no_grad():
        logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

In [15]:
# Store predicted calues
predicted_ids = torch.argmax(logits, dim=-1)

In [16]:
print("Prediction:", processor.batch_decode(predicted_ids))


Prediction: ['mückenstiche sollte man nicht aufkratzen', 'ist diese leitung sicher', 'die ratten verlassen das segene schif', 'ich habe eine neue arbeit', 'was sieht kamera eins gerade', 'was für einen angeber dachte horst im stillen', 'rückgängig machen', 'war die intigration erfolgreich', 'welche distripition mit klarzeit unterstützungen', 'lasst uns noch pabrika in den salat schnipseln']


In [12]:
print("Reference:", test_dataset[:10]["sentence"])


Reference: ['Mückenstiche sollte man nicht aufkratzen.', 'Ist diese Leitung sicher?', 'Die Ratten verlassen das sinkende Schiff.', 'Ich habe eine neue Arbeit.', 'Was sieht Kamera Eins gerade?', '"Was für ein Angeber", dachte Horst im Stillen.', 'Rückgängig machen.', 'War die Integration erfolgreich?', 'Welche Distribution mit Langzeitunterstützung nutzt du am liebsten?', 'Lass uns noch Paprika in den Salat schnipseln.']
